# Tuning GNN-ePC-SAFT

## Starting point

In [ ]:
import os, shutil
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("awskeyid")
secret_value_1 = user_secrets.get_secret("awssecretkey")
secret_value_2 = user_secrets.get_secret("ghtk")
secret_value_3 = user_secrets.get_secret("kaggle")
secret_value_4 = user_secrets.get_secret("username")
secret_value_5 = user_secrets.get_secret("wandbkey")
secret_value_6 = user_secrets.get_secret("email")

os.environ["KAGGLE_USERNAME"] = secret_value_4
os.environ["KAGGLE_KEY"] = secret_value_3
os.environ["WANDB_API_KEY"] = secret_value_5
os.environ["AWS_ACCESS_KEY_ID"] = secret_value_0
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_value_1
os.environ['PATH'] = "/root/.local/bin:" + os.environ['PATH']

In [ ]:
%%bash
python -m pip install --upgrade pip -q

pip install pipenv --user -q


In [ ]:
%%bash
# for the cloud 
rm -rf .* *
git clone --branch dev --single-branch -q https://{secret_value_2}@github.com/wildsonbbl/gnnepcsaft.git .

In [ ]:
%%capture req_output --no-stdout
%%bash
pipenv install -r requirements.txt -q
echo "\nFinished pip installing requirements.txt\n"
pipenv install torch-scatter
echo "\nFinished pip installing torch-scatter\n"

# install pcsaft
curl -O -L https://gitlab.com/libeigen/eigen/-/archive/master/eigen-master.zip
curl -O -L https://github.com/zmeri/PC-SAFT/archive/refs/tags/v1.4.1.zip
unzip -q eigen-master.zip
unzip -q v1.4.1.zip
cp -rf eigen-master/. PC-SAFT-1.4.1/externals/eigen
pipenv install ./PC-SAFT-1.4.1 -q
# install pcsaft
echo "\nFinished pip installing pcsaft\n"


In [ ]:
%%bash

echo AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID > .env
echo AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY >>.env
echo WANDB_API_KEY=$WANDB_API_KEY >> .env

In [ ]:
%%bash
pipenv run dvc pull -f -q

In [ ]:
# previous running results
# !cp -a /kaggle/input/gnnepcsaft/tuning_b3/root/* ~/

In [ ]:
%%bash
pipenv run pip show pcsaft | grep -E "Version|Name"

lscpu | grep -E "Model name|Core|core" 

nvidia-smi

## Tuning

### Tuning run

In [ ]:
%%bash
pipenv run python -m gnnepcsaft.train.tuner \
--workdir=/kaggle/working/gnnepcsaft --config=gnnepcsaft/configs/default.py \
--dataset=ramirez --tags="batch 3","v2" \
--num_cpu=0.75 --num_cpu_trainer=0.25 --num_gpus=0.25 \
--num_samples=64 --time_budget_s=39600 --verbose=0 \
--config.num_train_steps=25000 --config.eval_every_steps=5000 \
--config.model=PNAL --config.batch_size=512 \
--config.job_type="tuning" \
--get_result=False #--restoredir=/root/ray_results/gnnpcsaft/

### Clear workspace

In [ ]:
!rm -rf .* *

### Zip results

In [ ]:
%%bash
rm -rf ~/ray*/gnn*/Torch*/check* ~/ray*/gnn*/Torch*/wandb
zip -r tuning.zip ~/ray* -q

In [ ]:
req_output.show()